# reading data

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    ! wget https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py -O setup_google_colab.py
    import setup_google_colab
    setup_google_colab.setup_week1() 
    
import sys
sys.path.append("..")
from common.download_utils import download_week1_resources

download_week1_resources()

--2020-09-09 18:56:42--  https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1939 (1.9K) [text/plain]
Saving to: ‘setup_google_colab.py’

setup_google_colab. 100%[===================>]   1.89K  --.-KB/s    in 0s      

2020-09-09 18:56:42 (28.7 MB/s) - ‘setup_google_colab.py’ saved [1939/1939]



# Text preprocessing

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from ast import literal_eval
import pandas as pd
import numpy as np

In [ ]:
def read_data(filename):
    data = pd.read_csv(filename, sep='\t')
    data['tags'] = data['tags'].apply(literal_eval)
    return data

In [ ]:
train = read_data('data/train.tsv')
validation = read_data('data/validation.tsv')
test = pd.read_csv('data/test.tsv', sep='\t')

In [ ]:
train.head()

,title,tags
0,How to draw a stacked dotplot in R?,[r]
1,mysql select all records where a datetime fiel...,"[php, mysql]"
2,How to terminate windows phone 8.1 app,[c#]
3,get current time in a specific country via jquery,"[javascript, jquery]"
4,Configuring Tomcat to Use SSL,[java]


In [ ]:
X_train, y_train = train['title'].values, train['tags'].values
X_val, y_val = validation['title'].values, validation['tags'].values
X_test = test['title'].values

In [ ]:
print(X_train[0],y_train[0])

How to draw a stacked dotplot in R? ['r']


In [ ]:
import re

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def text_prepare(text):
    """
        text: a string
        
        return: modified initial string
    """
    # converting text into lowercase
    text = text.lower()
    #text = # replace REPLACE_BY_SPACE_RE symbols by space in text
    text=  REPLACE_BY_SPACE_RE.sub(' ',text)
    #text = # delete symbols which are in BAD_SYMBOLS_RE from text
    text= ''.join(BAD_SYMBOLS_RE.split(text))
    #text = # delete stopwords from text
    text= ' '.join(words for words in text.split() if not words in STOPWORDS)
    return text

In [ ]:
X_train = [text_prepare(x) for x in X_train]
X_val = [text_prepare(x) for x in X_val]
X_test = [text_prepare(x) for x in X_test]

In [ ]:
X_train[:3]

['draw stacked dotplot r',
 'mysql select records datetime field less specified value',
 'terminate windows phone 81 app']

# counting of words and tags

In [ ]:
# Dictionary of all tags from train corpus with their counts.
tags_counts = {}
# Dictionary of all words from train corpus with their counts.
words_counts = {}
from tqdm import tqdm
tokenizer = nltk.tokenize.TreebankWordTokenizer()
words=[]
for i in tqdm(range(len(X_train))):
  word = tokenizer.tokenize(X_train[i])
  words.append(word)
def freq_counts(words):
  dic_words={}
  for i in tqdm(range(len(words))):
    for y in set(words[i]):
      if y in dic_words:
        dic_words[y]=dic_words[y] + words[i].count(y)
      else:
        dic_words[y]=words[i].count(y)
  return dic_words


100%|██████████| 100000/100000 [00:05<00:00, 17100.60it/s]


In [ ]:
tags_counts = freq_counts(y_train)
words_counts= freq_counts(words)


100%|██████████| 100000/100000 [00:00<00:00, 285279.06it/s]


In [ ]:
print(tags_counts)

{'r': 1727, 'mysql': 3092, 'php': 13907, 'c#': 19077, 'jquery': 7510, 'javascript': 19078, 'java': 18661, 'ruby-on-rails': 3344, 'ruby': 2326, 'ruby-on-rails-3': 692, 'json': 2026, 'spring': 1346, 'spring-mvc': 618, 'codeigniter': 786, 'class': 509, 'ios': 3256, 'html': 4668, 'eclipse': 992, 'c++': 6469, 'python': 8940, 'list': 693, 'objective-c': 4338, 'swift': 1465, 'xaml': 438, 'asp.net': 3939, 'multithreading': 1118, 'wpf': 1289, 'image': 672, 'performance': 512, 'twitter-bootstrap': 501, 'linq': 964, 'xml': 1347, 'numpy': 502, 'ajax': 1767, 'django': 1835, 'laravel': 525, 'android': 2818, 'rest': 456, 'asp.net-mvc': 1244, 'web-services': 633, 'string': 1573, 'excel': 443, 'winforms': 1468, 'arrays': 2277, 'sockets': 579, 'c': 3119, 'osx': 490, 'entity-framework': 649, 'mongodb': 350, 'opencv': 401, 'xcode': 900, 'uitableview': 460, 'algorithm': 419, 'python-2.7': 421, 'angularjs': 1353, 'dom': 400, 'swing': 759, '.net': 3872, 'vb.net': 1918, 'google-maps': 408, 'hibernate': 807, '

In [ ]:
most_common_tags = sorted(tags_counts.items(), key=lambda x: x[1], reverse=True)[:3]
most_common_words = sorted(words_counts.items(), key=lambda x: x[1], reverse=True)[:3]
print(most_common_tags,most_common_words)

[('javascript', 19078), ('c#', 19077), ('java', 18661)] [('using', 8278), ('c', 5800), ('php', 5614)]


# Bag of words

In [ ]:

DICT_SIZE = 5000
WORDS_TO_INDEX={}
popular_words = sorted(words_counts.items(),key= lambda x:x[1],reverse=True)[:5000]
count = 0
for i in tqdm(popular_words):
  WORDS_TO_INDEX[i[0]]=count
  count = count+1
ALL_WORDS = WORDS_TO_INDEX.keys()

def my_bag_of_words(text, words_to_index, dict_size):
    """
        text: a string
        dict_size: size of the dictionary
        
        return a vector which is a bag-of-words representation of 'text'
    """
    result_vector = np.zeros(dict_size)
    words = tokenizer.tokenize(text)
    for i in words:
      if i in words_to_index:
        result_vector[words_to_index[i]] += 1
    return result_vector

100%|██████████| 5000/5000 [00:00<00:00, 801418.53it/s]


In [ ]:
from scipy import sparse as sp_sparse

In [ ]:
X_train_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_train])
X_val_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_val])
X_test_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_test])
print('X_train shape ', X_train_mybag.shape)
print('X_val shape ', X_val_mybag.shape)
print('X_test shape ', X_test_mybag.shape)

X_train shape  (100000, 5000)
X_val shape  (30000, 5000)
X_test shape  (20000, 5000)


# multilabel classifier using bag_of_words representation

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer


In [ ]:
mlb = MultiLabelBinarizer(classes=sorted(tags_counts))
tran_y_train = mlb.fit_transform(y_train)
tran_y_val = mlb.fit_transform(y_val)


In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier


In [ ]:
def train_classifier(X_train, y_train):
    """
      X_train, y_train — training data
      
      return: trained classifier
    """
    
    # Create and fit LogisticRegression wraped into OneVsRestClassifier.
    #LR = LogisticRegression()
    LR_model = LogisticRegression(n_jobs=-1,C=10)
    ORS = OneVsRestClassifier(LR_model,n_jobs=100)
    clf = ORS.fit(X_train,y_train)

    ######################################
    ######### YOUR CODE HERE #############
    ######################################    
    return clf

In [ ]:
classifier_mybag = train_classifier(X_train_mybag, tran_y_train)


In [ ]:
y_val_predicted_labels_mybag = classifier_mybag.predict(X_val_mybag)
y_val_predicted_scores_mybag = classifier_mybag.decision_function(X_val_mybag)

In [ ]:
y_val_pred_inversed = mlb.inverse_transform(y_val_predicted_labels_mybag)
y_val_inversed = mlb.inverse_transform(tran_y_val)
for i in range(5):
    print('Title:\t{}\nTrue labels:\t{}\nPredicted labels:\t{}\n\n'.format(
        X_val[i],
        ','.join(y_val_inversed[i]),
        ','.join(y_val_pred_inversed[i])
    ))

Title:	odbc_exec always fail
True labels:	php,sql
Predicted labels:	


Title:	access base classes variable within child class
True labels:	javascript
Predicted labels:	


Title:	contenttype application json required rails
True labels:	ruby,ruby-on-rails
Predicted labels:	ruby-on-rails


Title:	sessions sinatra used pass variable
True labels:	ruby,session
Predicted labels:	ruby


Title:	getting error type json exist postgresql rake db migrate
True labels:	json,ruby,ruby-on-rails
Predicted labels:	json,ruby-on-rails




In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score

In [ ]:
def print_evaluation_scores(y_val, predicted):
    
   
    acc = accuracy_score(y_val,predicted)
    f1 = f1_score(y_val,predicted,average='weighted')
    precision = average_precision_score(y_val,predicted,average='weighted')
    print('Accuracy :',acc,'\nf1-score :',f1,'\nprecision :',precision)
    return acc,f1,precision


In [ ]:
print('Bag-of-words')
print_evaluation_scores(tran_y_val, y_val_predicted_labels_mybag)

Bag-of-words


(0.3565, 0.6476067894259262, 0.5094770243402924)

# Trying few hyper-parameters

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = [0.1,1,10,100]
f1score={}
for i in parameters:
  ORS = OneVsRestClassifier(LogisticRegression(C=i),n_jobs=100)
  clf = ORS.fit(X_train_mybag, tran_y_train)
  pred_train = clf.predict(X_train_mybag)
  pred_val = clf.predict(X_val_mybag)
  print('evalation value with c = ',i,'on training set')
  print_evaluation_scores(tran_y_train,pred_train)
  print('\nevalation value with c = ',i,'on validation set')
  print_evaluation_scores(tran_y_val,pred_val)
  print('-'*40)


 

evalation value with c =  0.1 on training set
Accuracy : 0.32385 
f1-score : 0.6025104691689465 
precision : 0.47521572702893905

evalation value with c =  0.1 on validation set
Accuracy : 0.3100333333333333 
f1-score : 0.5909545305138711 
precision : 0.4635436180297992
----------------------------------------
evalation value with c =  1 on training set
Accuracy : 0.41132 
f1-score : 0.6913585085857322 
precision : 0.5590964663282852

evalation value with c =  1 on validation set
Accuracy : 0.3565 
f1-score : 0.6476067894259262 
precision : 0.5094770243402924
----------------------------------------
evalation value with c =  10 on training set
Accuracy : 0.48932 
f1-score : 0.7546106056533863 
precision : 0.6263761155716601

evalation value with c =  10 on validation set
Accuracy : 0.34836666666666666 
f1-score : 0.6506978120145793 
precision : 0.5004479762717012
----------------------------------------
evalation value with c =  100 on training set
Accuracy : 0.52274 
f1-score : 0.7815

In [ ]:
# c= 10 works best for the bags of words representation

# lets build model using tfidf representation


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
def tfidf_features(X_train, X_val, X_test):
    """
        X_train, X_val, X_test — samples        
        return TF-IDF vectorized representation of each sample and vocabulary
    """
    # Create TF-IDF vectorizer with a proper parameters choice
    # Fit the vectorizer on the train set
    # Transform the train, test, and val sets and return the result
    
    
    tfidf_vectorizer = TfidfVectorizer(min_df=5,max_df=0.9,ngram_range=(1,3),token_pattern='(\S+)')
    tfidf_vectorizer.fit(X_train)
    X_train = tfidf_vectorizer.transform(X_train)
    X_val = tfidf_vectorizer.transform(X_val)
    X_test = tfidf_vectorizer.transform(X_test)
    
    
    
    return X_train, X_val, X_test, tfidf_vectorizer.vocabulary_

In [ ]:
X_train_tfidf, X_val_tfidf, X_test_tfidf, tfidf_vocab = tfidf_features(X_train, X_val, X_test)
tfidf_reversed_vocab = {i:word for word,i in tfidf_vocab.items()}

In [ ]:
classifier_tfidf = train_classifier(X_train_tfidf, tran_y_train)


In [ ]:
y_val_predicted_labels_tfidf = classifier_tfidf.predict(X_val_tfidf)
y_train_predicted_labels_tfidf = classifier_tfidf.predict(X_train_tfidf)

y_val_predicted_scores_tfidf = classifier_tfidf.decision_function(X_val_tfidf)
y_train_predicted_scores_tfidf = classifier_tfidf.decision_function(X_train_tfidf)

In [ ]:
y_val_pred_inversed = mlb.inverse_transform(y_val_predicted_labels_tfidf)
y_val_inversed = mlb.inverse_transform(tran_y_val)
for i in range(5):
    print('Title:\t{}\nTrue labels:\t{}\nPredicted labels:\t{}\n\n'.format(
        X_val[i],
        ','.join(y_val_inversed[i]),
        ','.join(y_val_pred_inversed[i])
    ))

Title:	odbc_exec always fail
True labels:	php,sql
Predicted labels:	


Title:	access base classes variable within child class
True labels:	javascript
Predicted labels:	


Title:	contenttype application json required rails
True labels:	ruby,ruby-on-rails
Predicted labels:	ruby-on-rails


Title:	sessions sinatra used pass variable
True labels:	ruby,session
Predicted labels:	ruby


Title:	getting error type json exist postgresql rake db migrate
True labels:	json,ruby,ruby-on-rails
Predicted labels:	json,ruby-on-rails




In [ ]:
print('Tfidf on training set')
print_evaluation_scores(tran_y_train, y_train_predicted_labels_tfidf)
print('\nTfidf on validation set')
print_evaluation_scores(tran_y_val, y_val_predicted_labels_tfidf)

Tfidf on training set
Accuracy : 0.52229 
f1-score : 0.7783352691859953 
precision : 0.6622472137820092

Tfidf on validation set
Accuracy : 0.3631666666666667 
f1-score : 0.6541861123543633 
precision : 0.5103444960640318


(0.3631666666666667, 0.6541861123543633, 0.5103444960640318)

# using tfidf we achieved higher accuracy than bags og words

### Analysis of the most important features

In [ ]:
def print_words_for_tag(classifier, tag, tags_classes, index_to_words, all_words):
    """
        classifier: trained classifier
        tag: particular tag
        tags_classes: a list of classes names from MultiLabelBinarizer
        index_to_words: index_to_words transformation
        all_words: all words in the dictionary
        
        return nothing, just print top 5 positive and top 5 negative words for current tag
    """
    print('Tag:\t{}'.format(tag))
    
    # Extract an estimator from the classifier for the given tag.
    # Extract feature coefficients from the estimator. 
  
    index = tags_classes.index(tag)
    est = classifier.estimators_[index]
    feature = est.coef_
    score_of_words = {}
    for number in index_to_words:
      score_of_words[index_to_words[number]]=feature[0][number]
    
    positive = sorted(score_of_words.items(),key= lambda x:x[1],reverse=True)[:5]
    top_positive_words =[]
    for word in positive:
      top_positive_words.append(word[0])
    negative = sorted(score_of_words.items(),key= lambda x:x[1],reverse=False)[:5]
    top_negative_words=[]
    for word in negative:
      top_negative_words.append(word[0])
    print('Top positive words:\t{}'.format(', '.join(top_positive_words)))
    print('Top negative words:\t{}\n'.format(', '.join(top_negative_words)))

In [ ]:
print_words_for_tag(classifier_tfidf, 'c', mlb.classes, tfidf_reversed_vocab, ALL_WORDS)
print_words_for_tag(classifier_tfidf, 'c++', mlb.classes, tfidf_reversed_vocab, ALL_WORDS)
print_words_for_tag(classifier_tfidf, 'python', mlb.classes, tfidf_reversed_vocab, ALL_WORDS)

Tag:	c
Top positive words:	c, malloc, scanf, fscanf, printf
Top negative words:	php, java, python, javascript, c#

Tag:	c++
Top positive words:	c++, qt, boost, mfc, stdstring
Top negative words:	java, php, python, javascript, c#

Tag:	python
Top positive words:	python, pandas, numpy, matplotlib, tkinter
Top negative words:	php, c#, java, r, javascript

